Ranking system for Hyper Duplex Style Alloys

In [1]:
# Imports
import os
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Mn Standard
Cr_values  = [27.0, 27.5, 28.0, 28.5, 29.0]
N_values   = [0.00, 0.05, 0.10, 0.15]
C_values = [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.10, 0.30, 0.50, 0.70]
Mn_values = [9.5, 10.0, 10.5]

X_values   = [0.0, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0, 5.5, 6.0, 6.5, 7.0, 7.5, 8.0, 8.5, 9.0, 9.5, 10.0]

In [3]:
# Input data 
X_elements = ["Ni", "Mo", "Si"]

service_temperature_C = 20

""" https://en.wikipedia.org/wiki/Prices_of_chemical_elements """
cost_usd_per_kg = {
    "Fe": 0.42,
    "C":  0.13,
    "Cr": 9.4,
    "Mo": 40.1,
    "Ni": 13.9,
    "Mn": 1.8,
    "Si": 1.7,
    "N":  0.15,
}

density_proxy = 7.8

In [4]:
# Ranking calculation proxies

def pren(Cr_wt, Mo_wt=0.0, N_wt=0.0, W_wt=0.0):
    """PREN ≈ %Cr + 3.3(%Mo + 0.5%W) + 16%N  (Si does not enter PREN)"""
    pren = Cr_wt + 3.3*(Mo_wt + 0.5*W_wt) + 16.0*N_wt
    return pren

# CHANGE FUNCTION
def strength_proxy(alloy):
    """Solid solution strengthening proxy (MPa)"""
    k = {"Cr": 35.0, "Mo": 120.0, "Ni": 20.0, "Mn": 25.0, "Si": 20.0, "C": 250.0, "N": 180.0}
    s = 0.0
    for el, wt in alloy.items():
        if el in k and wt > 0:
            s += k[el] * math.sqrt(wt)
    return s

def cost_proxy(alloy):
    """Cost proxy ($/kg)"""
    cost = 0.0
    for el, wt in alloy.items():
        el_cost = cost_usd_per_kg.get(el, 0.0)
        cost += (wt / 100.0) * el_cost
    return cost

In [5]:
# Generate data
rows = []
for X in X_elements:
    for C in C_values:
        for Cr in Cr_values:
            for N in N_values:
                for Mn in Mn_values:
                    for x_wt in X_values:
                        comp = {"C": C, "Cr": Cr, "N": N, "Mn": Mn, X: x_wt}
                        PREN = pren(Cr_wt=Cr, Mo_wt=comp.get("Mo", 0.0), N_wt=N)
                        dss  = strength_proxy(comp)

                        Fe_balance = 100.0 - (C + Cr + N + Mn + x_wt)

                        rows.append({
                            "X_element": X,
                            "C_wt%": C,
                            "Cr_wt%": Cr,
                            "N_wt%": N,
                            "Mn_wt%": Mn,
                            f"{X}_wt%": x_wt,
                            "Fe_wt% (balance)": Fe_balance,
                            "PREN": PREN,
                            "Strength_proxy_MPa": dss,
                            "Cost_proxy_$_per_kg": cost_proxy(comp),
                            "Service_T_C (info)": service_temperature_C,
                        })

df = pd.DataFrame(rows)

csv_path = "HD_Mn_Screening.csv"
df.to_csv(csv_path, index=False)
print(f"CSV saved: {csv_path}")
df.head(-10)

CSV saved: HD_Mn_Screening.csv


,X_element,C_wt%,Cr_wt%,N_wt%,Mn_wt%,Ni_wt%,Fe_wt% (balance),PREN,Strength_proxy_MPa,Cost_proxy_$_per_kg,Service_T_C (info),Mo_wt%,Si_wt%
0,Ni,0.01,27.0,0.00,9.5,0.0,63.49,27.0,283.920510,2.709013,20,NaN,NaN
1,Ni,0.01,27.0,0.00,9.5,0.5,62.99,27.0,298.062645,2.778513,20,NaN,NaN
2,Ni,0.01,27.0,0.00,9.5,1.0,62.49,27.0,303.920510,2.848013,20,NaN,NaN
3,Ni,0.01,27.0,0.00,9.5,1.5,61.99,27.0,308.415407,2.917513,20,NaN,NaN
4,Ni,0.01,27.0,0.00,9.5,2.0,61.49,27.0,312.204781,2.987013,20,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
49125,Si,0.70,29.0,0.15,10.5,NaN,56.65,31.4,583.009750,2.967135,20,NaN,3.0
49126,Si,0.70,29.0,0.15,10.5,NaN,56.15,31.4,585.785308,2.975635,20,NaN,3.5
49127,Si,0.70,29.0,0.15,10.5,NaN,55.65,31.4,588.368734,2.984135,20,NaN,4.0
49128,Si,0.70,29.0,0.15,10.5,NaN,55.15,31.4,590.795141,2.992635,20,NaN,4.5


In [6]:
df["Score"] = df["PREN"] + df["Strength_proxy_MPa"] / 100.0
df_sorted = df.sort_values(by="Score", ascending=False)
df_sorted.head(20)

,X_element,C_wt%,Cr_wt%,N_wt%,Mn_wt%,Ni_wt%,Fe_wt% (balance),PREN,Strength_proxy_MPa,Cost_proxy_$_per_kg,Service_T_C (info),Mo_wt%,Si_wt%,Score
32759,Mo,0.7,29.0,0.15,10.5,NaN,49.65,64.4,927.842053,6.926135,20,10.0,NaN,73.678421
32738,Mo,0.7,29.0,0.15,10.0,NaN,50.15,64.4,925.889736,6.917135,20,10.0,NaN,73.658897
32717,Mo,0.7,29.0,0.15,9.5,NaN,50.65,64.4,923.887969,6.908135,20,10.0,NaN,73.638880
31499,Mo,0.5,29.0,0.15,10.5,NaN,49.85,64.4,895.453742,6.925875,20,10.0,NaN,73.354537
31478,Mo,0.5,29.0,0.15,10.0,NaN,50.35,64.4,893.501425,6.916875,20,10.0,NaN,73.335014
31457,Mo,0.5,29.0,0.15,9.5,NaN,50.85,64.4,891.499658,6.907875,20,10.0,NaN,73.314997
32507,Mo,0.7,28.5,0.15,10.5,NaN,50.15,63.9,926.210154,6.879135,20,10.0,NaN,73.162102
32486,Mo,0.7,28.5,0.15,10.0,NaN,50.65,63.9,924.257837,6.870135,20,10.0,NaN,73.142578
32465,Mo,0.7,28.5,0.15,9.5,NaN,51.15,63.9,922.256071,6.861135,20,10.0,NaN,73.122561
30239,Mo,0.3,29.0,0.15,10.5,NaN,50.05,64.4,855.607686,6.925615,20,10.0,NaN,72.956077
